<h1 align=center> Segmentation and clustering Neighborhoods in Tornoto </h1>

## introduction
In this project we will explore neighbourhood in Tornoto using Forsqure API.
first we'll webscrap the data from wikipedia and then using ForSquare API and folium map extract imporatnt information from this data. We will use k-means clusterning algorithm to do this task

## Table of Contents
1.  <a href="#item1"> import required libraries</a>
2.  <a href="#item1">extract and Explore Dataset from Wikipedia using Webscrapping</a>


<h1 align='center'> Part-1 </h1>

## 1.Import required libraries

In [2]:
#import libraries
!pip install html5lib
!pip install lxml
!pip install geocoder
!pip install bs4
!pip install folium 
!pip install geopy
import folium
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import requests
import geocoder
print("all libraries installed and imported succefully")

You should consider upgrading via the 'c:\users\mahi7\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'c:\users\mahi7\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'c:\users\mahi7\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.
You should consider upgrading via the 'c:\users\mahi7\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.


all libraries installed and imported succefully


## 2.extract and Explore Dataset from Wikipedia using Webscrapping

In [3]:
# using webscrapping to extract data
url="https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
html_data= requests.get(url).text
beautiful_soup=BeautifulSoup(html_data,"html.parser")

# crete new data frame
df=pd.DataFrame(columns=['PostalCode','Borough','Neighborhood'])
df


# append data in data frame by webscrpping
for row in beautiful_soup.find("tbody").find_all("tr"):
    for col in row.find_all("td"):
        if col.span.text=='Not assigned':
            pass
        else:
            pcode=col.p.text[:3]
            borough=(col.span.text).split('(')[0]
            nhood=(((((col.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
            df=df.append({'PostalCode':pcode,'Borough':borough,'Neighborhood':nhood},ignore_index=True)
        
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})


In [4]:
df.shape


(103, 3)

<h1 align='center'> Part-2 </h1>

### Read coordinates in dataframe from csv files

In [5]:
df_coord=pd.read_csv("Geospatial_Coordinates.csv")
df_coord.rename(columns= {'Postal Code':'PostalCode'},inplace=True)


### merge both data frame to make final data frame which contain coordinates also

In [6]:
result=pd.merge(df,df_coord,on=['PostalCode'])
result

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto Business,Enclave of M4L,43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509


<h1 align='center'> Part-3 </h1>

## clustering and Segmenting 

You should consider upgrading via the 'c:\users\mahi7\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'c:\users\mahi7\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.


#### Use Geopy library to get coordinates of Toronto

In [22]:
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

address='Toronto'
geolocator=Nominatim(user_agent='Tr_explore')
location=geolocator.geocode(address)
latitude=location.latitude
longitude=location.longitude
print("The geographical coordinates of Tronoto are: {},{}".format(latitude,longitude))

The geographical coordinates of Tronoto are: 43.6534817,-79.3839347


###  Create a map of Toronto and add all neighborhoods on the same map

In [49]:
# create map of Tronto
map_toronto= folium.Map(location=[latitude,longitude],zoom_start=13)

#add neghborhood on toronto map

for lat,lng,borough,nhood in zip(result['Latitude'],result['Longitude'],result['Borough'],result['Neighborhood']):
        label="{},{}".format(nhood,borough)
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
        [lat,lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='red',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)

map_toronto


above map is too congested, so for illustration purpose We'll use small set of Data
Let's simplify the above map and consider only those neighborhood whose Borough name conatains "Toronto"

In [61]:
result.head()
tr_data=result[ result['Borough'].str.contains('Toronto')].reset_index(drop=True)
tr_data

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
3,M4E,East Toronto,The Beaches,43.676357,-79.293031
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
5,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
6,M6G,Downtown Toronto,Christie,43.669542,-79.422564
7,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.650571,-79.384568
8,M6H,West Toronto,"Dufferin, Dovercourt Village",43.669005,-79.442259
9,M4J,East York/East Toronto,The Danforth East,43.685347,-79.338106


In [59]:
# create map of Tronto
map_toronto= folium.Map(location=[latitude,longitude],zoom_start=13)

#add neghborhood on toronto map

for lat,lng,borough,nhood in zip(tr_data['Latitude'],tr_data['Longitude'],tr_data['Borough'],tr_data['Neighborhood']):
        label="{},{}".format(nhood,borough)
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
        [lat,lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='red',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)

map_toronto

Now we are going to use Foursqure API to explore the neighborhood and explore them

### Define Foursqure credential

In [60]:
CLIENT_ID = '15VSS0FQMGGTRNLEKHI1SFVW1CCLK12MDZ3E4GZKWJHNQHO1' # your Foursquare ID
CLIENT_SECRET = 'FLYBN5UVHBPC1DZGXGJWCI1K53K3PJHVCYXGMQJVE4FJPQOC' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 15VSS0FQMGGTRNLEKHI1SFVW1CCLK12MDZ3E4GZKWJHNQHO1
CLIENT_SECRET:FLYBN5UVHBPC1DZGXGJWCI1K53K3PJHVCYXGMQJVE4FJPQOC


Let's explore the first neighborhood from our data frame

In [70]:
nhood_name=tr_data.loc[0,'Neighborhood'].split(',')[0].strip()
nhood_name

'Regent Park'

Get the neighborhood latitude and longitude

In [72]:
nhood_lat= tr_data.loc[0,'Latitude']
nhood_lng= tr_data.loc[0,'Longitude']
print("selected neighborhood is {} and its Coordinate are: {},{}".format(nhood_name,nhood_lat,nhood_lng))

selected neighborhood is Regent Park and its Coordinate are: 43.6542599,-79.3606359


#### Let's get top 100 venues taht are in Regent Park within a radius of 300 metre

In [74]:
LIMIT=100
radius=300
url="https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    nhood_lat, 
    nhood_lng, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?client_id=15VSS0FQMGGTRNLEKHI1SFVW1CCLK12MDZ3E4GZKWJHNQHO1&client_secret=FLYBN5UVHBPC1DZGXGJWCI1K53K3PJHVCYXGMQJVE4FJPQOC&v=20180605&ll=43.6542599,-79.3606359&radius=300&limit=100'

In [80]:
results=requests.get(url).json()
results


In [86]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [88]:
venue=results['response']['groups'][0]['items']
nearby_venues=json_normalize(venue)
nearby_venues

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]


# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

print(nearby_venues.shape)
nearby_venues.head()

(16, 4)


<ipython-input-88-6f98163cd504>:2: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  nearby_venues=json_normalize(venue)


,name,categories,lat,lng
0,Roselle Desserts,Bakery,43.653447,-79.362017
1,Tandem Coffee,Coffee Shop,43.653559,-79.361809
2,Cooper Koo Family YMCA,Distribution Center,43.653249,-79.358008
3,Body Blitz Spa East,Spa,43.654735,-79.359874
4,Morning Glory Cafe,Breakfast Spot,43.653947,-79.361149


# Explore Neighborhoods 

In [111]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)
    

In [112]:
toronto_venue=getNearbyVenues(tr_data['Neighborhood'],tr_data['Latitude'],tr_data['Longitude'])

Regent Park, Harbourfront
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
The Danforth  East
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West
High Park, The Junction South
North Toronto West
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Rosedale
Enclave of M5E
St. James Town, Cabbagetown
First Canadi

In [113]:
print(toronto_venue.shape)
toronto_venue.head()

(1588, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Regent Park, Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Regent Park, Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
3,"Regent Park, Harbourfront",43.65426,-79.360636,Impact Kitchen,43.656369,-79.356980,Restaurant
4,"Regent Park, Harbourfront",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa


###### Count No. of values returned for each neighborhood

In [117]:
toronto_venue.groupby("Neighborhood").count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,59,59,59,59,59,59
"Brockton, Parkdale Village, Exhibition Place",24,24,24,24,24,24
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",15,15,15,15,15,15
Central Bay Street,62,62,62,62,62,62
Christie,16,16,16,16,16,16


Let's try to find out no. of venues for each category in neighborhood of Toronto <br>
We can sort them is descending order and we can find that coffe shop is maximum followed by cafe an then Restaurant

In [129]:
toronto_venue.groupby("Venue Category").count().sort_values(by='Venue',ascending=False)

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude
Venue Category,,,,,,
Coffee Shop,153,153,153,153,153,153
Café,82,82,82,82,82,82
Restaurant,51,51,51,51,51,51
Italian Restaurant,41,41,41,41,41,41
Hotel,40,40,40,40,40,40
...,...,...,...,...,...,...
Health Food Store,1,1,1,1,1,1
Health & Beauty Service,1,1,1,1,1,1
Harbor / Marina,1,1,1,1,1,1


## Analyze each Neighborhood